In [1]:
import os
import sys
import time

verbose  = os.environ.get('RADICAL_PILOT_VERBOSE', 'REPORT')
os.environ['RADICAL_PILOT_VERBOSE'] = verbose

import radical.pilot as rp
import radical.utils as ru

from rose.learner import ReinforcementLearner
from rose.engine import Task, ResourceEngine
from rose.metrics import GREATER_THAN_THRESHOLD

In [2]:
engine = ResourceEngine({'runtime': 30,
                         'resource': 'local.localhost'})
rl = ReinforcementLearner(engine)
code_path = f'{sys.executable} {os.getcwd()}'

Resource Engine started successfully

Task 'environment' ready to submit; resolved dependencies: []
submitting ['environment'] for execution
task.000000 is DONE
Task 'update' ready to submit; resolved dependencies: []
submitting ['update'] for execution
task.000001 is DONE
Task 'environment' ready to submit; resolved dependencies: []
submitting ['environment'] for execution
task.000002 is DONE
Task 'update' ready to submit; resolved dependencies: []
submitting ['update'] for execution
task.000003 is DONE
Task 'environment' ready to submit; resolved dependencies: []
submitting ['environment'] for execution
task.000004 is DONE
Task 'update' ready to submit; resolved dependencies: []
submitting ['update'] for execution
task.000005 is DONE
Task 'environment' ready to submit; resolved dependencies: []
submitting ['environment'] for execution
task.000006 is DONE
Task 'update' ready to submit; resolved dependencies: []
submitting ['update'] for execution
task.000007 is DONE
Task 'environment'

In [3]:
# Define and register the environment task
@rl.environment_task
def environment(*args):
    return Task(executable=f'{code_path}/environment.py', arguments=args)

# Define and register the policy update task
@rl.update_task
def update(*args):
    return Task(executable=f'{code_path}/update.py', arguments=args)

@rl.as_stop_criterion(metric_name='REWARD', threshold=0.001, operator=GREATER_THAN_THRESHOLD)
def check_reward(*args):
    return Task(executable=f'{code_path}/check_reward.py', arguments=args)

In [4]:
# Custom teaching loop with reinforcement Learning
def teach():
    for iter in range(20):
        env = environment()
        env.result()
        upd = update()
        upd.result()
# invoke the custom/user-defined teach() method
teach()

Registered task 'environment' and id of 000000 with dependencies: []
Registered task 'update' and id of 000001 with dependencies: []
Registered task 'environment' and id of 000002 with dependencies: []
Registered task 'update' and id of 000003 with dependencies: []
Registered task 'environment' and id of 000004 with dependencies: []
Registered task 'update' and id of 000005 with dependencies: []
Registered task 'environment' and id of 000006 with dependencies: []
Registered task 'update' and id of 000007 with dependencies: []
Registered task 'environment' and id of 000008 with dependencies: []
Registered task 'update' and id of 000009 with dependencies: []
Registered task 'environment' and id of 000010 with dependencies: []
Registered task 'update' and id of 000011 with dependencies: []
Registered task 'environment' and id of 000012 with dependencies: []
Registered task 'update' and id of 000013 with dependencies: []
Registered task 'environment' and id of 000014 with dependencies: []


In [10]:
engine.shutdown()

Shutdown is triggered, terminating the resources gracefully
